In [ ]:
import numpy as np
import sys
import warnings
import time
import pandas as pd
from sklearn.datasets import fetch_openml

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
# --- COMPETITORS ---
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
try:
    from xgboost import XGBClassifier
    from lightgbm import LGBMClassifier
except ImportError:
    print(">>> [WARNING] XGBoost or LightGBM not installed. Benchmarks will be limited.")

import numpy as np
import sys
import warnings
import time
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# --- GPU ACTIVATION ---
try:
    import cupy as cp
    print(">>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.")
    cp.random.seed(42)
except ImportError:
    print(">>> [FATAL] GPU REQUIRED FOR TITAN.")
    sys.exit()

warnings.filterwarnings("ignore")

# ==========================================
# 1. THE MEMORY UNIT (The Synapse)
# ==========================================
class MEMORY_UNIT:
    def __init__(self, n_neurons, input_dim, seed, activation_func):
        cp.random.seed(seed)
        self.n_neurons = n_neurons
        self.activation = activation_func

        # 1. Input Weights (The Sensory Cortex)
        self.W_in = cp.random.uniform(-1.0, 1.0, (n_neurons, input_dim)).astype(cp.float32)

        # 2. Internal Weights (The Thinking Cortex)
        # Orthogonal initialization for stability (Bruce Lee Balance)
        self.W_cortex = cp.random.normal(0, 1.0 / np.sqrt(n_neurons), (n_neurons, n_neurons)).astype(cp.float32)
        try:
            # Spectral Normalization: Keeps the echo chamber stable
            u, s, v = cp.linalg.svd(self.W_cortex)
            self.W_cortex = self.W_cortex / s[0]
        except: pass

        self.readout = None

    def process(self, X_gpu):
        # The Echo State Loop
        state = cp.zeros((X_gpu.shape[0], self.n_neurons), dtype=cp.float32)
        # Deep Thinking: 3 Recurrent Steps
        for _ in range(3):
            pre = (X_gpu @ self.W_in.T) + (state @ self.W_cortex.T)
            state = self.activation(pre)
        return state

    def fit_residual(self, X_gpu, Residual_gpu, alpha=10.0):
        H = self.process(X_gpu)
        I = cp.eye(self.n_neurons, dtype=cp.float32)
        # Ridge Regression (The Solver)
        self.readout = cp.linalg.solve(H.T @ H + alpha*I, H.T @ Residual_gpu)

    def predict_correction(self, X_gpu):
        H = self.process(X_gpu)
        return H @ self.readout


# ==========================================
# 0. THE RESONANCE SCANNER (God's Eye)
# ==========================================
class RESONANCE_SCANNER:
    def __init__(self):
        self.activations = {}
        self._generate_functions()

    def _generate_functions(self):
        # 1. Standard
        self.activations['tanh'] = lambda x: cp.tanh(x)
        self.activations['swish'] = lambda x: x * (1 / (1 + cp.exp(-x)))
        self.activations['mish'] = lambda x: x * cp.tanh(cp.log(1 + cp.exp(x)))

        # 2. Fibonacci Frequencies (Nature's Math)
        fib = [0.5, 1.0, 1.618, 2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 34.0]
        for f in fib:
            self.activations[f'sin_phi_{f}'] = lambda x, freq=f: cp.sin(freq * x)
            self.activations[f'cos_phi_{f}'] = lambda x, freq=f: cp.cos(freq * x)

        # 3. Gaussian Kernels (Focus)
        for s in [0.5, 1.0, 5.0]:
            self.activations[f'rbf_{s}'] = lambda x, sig=s: cp.exp(-cp.square(x)/(2*sig**2))

    def scan(self, X_gpu, Residual_gpu, n_probe=200):
        best_name = "tanh"
        best_score = -np.inf

        # Fast Probe Projection
        cp.random.seed(int(time.time()))
        W_probe = cp.random.uniform(-1.0, 1.0, (n_probe, X_gpu.shape[1])).astype(cp.float32)
        Projected_X = X_gpu @ W_probe.T

        # print(f"   [SCANNER] Testing {len(self.activations)} Quantum Functions...", end="")

        for name, func in self.activations.items():
            try:
                H = func(Projected_X)
                # Quick Solver
                w = cp.linalg.solve(H.T @ H + 1.0*cp.eye(n_probe), H.T @ Residual_gpu)
                pred = H @ w

                # Score = Energy Reduction
                error = cp.linalg.norm(Residual_gpu - pred)
                if -error > best_score:
                    best_score = -error
                    best_name = name
            except: continue

        # print(f" Detected: '{best_name.upper()}'")
        return best_name, self.activations[best_name]


# ==========================================
# 2. TITAN INF (BRUCE LEE EDITION)
# ==========================================
class TITAN_INF(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.memory_vault = []
        self.base_model = None
        self.scanner = RESONANCE_SCANNER()

    def _tournament_selection(self, X, y):
        # Simulating the G.O.D. Protocol selection
        print(f" > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...")
        # (Simplified for brevity - assumes ExtraTrees is usually robust)
        model = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42, bootstrap=False)
        score = np.mean(cross_val_score(model, X, y, cv=3))
        print(f" > [VICTORY] Champion Selected: EXTRATREES (Base Acc: {score:.4%})")
        return model

    def perceive(self, X, y):
        self.input_dim = X.shape[1]
        self.n_classes = len(np.unique(y))

        # 1. Base Model (The Foundation)
        self.base_model = self._tournament_selection(X, y)
        self.base_model.fit(X, y)

        # 2. Generate Residuals (The Truth Gap)
        # We use Out-of-Bag (OOB) or CV to find where the model is WRONG
        print(" > [TITAN] Generating Truth Residuals...")
        base_probs = cross_val_predict(self.base_model, X, y, cv=5, method='predict_proba', n_jobs=-1)

        # 3. Enter The Dojo (Evolution)
        self._fit_evolution(X, y, base_probs)

    def _fit_evolution(self, X, y, base_probs):
        # PREPARE GPU DATA
        X_gpu = cp.asarray(X, dtype=cp.float32)
        y_gpu = cp.asarray(y)

        # Soft Targets (Prevents Overconfidence/Explosion)
        Y_target = cp.zeros((len(y), self.n_classes), dtype=cp.float32) + 0.01
        for i in range(self.n_classes):
            Y_target[y_gpu==i, i] = 0.99

        # Initial State
        current_logits = cp.asarray(np.log(base_probs + 1e-9))
        best_acc = accuracy_score(y, np.argmax(base_probs, axis=1))

        print(f" > [TITAN] Dojo Opened. Base Accuracy: {best_acc:.4%}")
        print(" > [STRATEGY] Bruce Lee Mode: Infinite Variations (Noise Injection)")

        # --- THE OMEGA LOOP ---
        cycle = 0
        patience = 0
        MAX_PATIENCE = 15

        while patience < MAX_PATIENCE:
            cycle += 1

            # 1. Calculate Error (The Gap)
            probs = cp.exp(current_logits) / cp.sum(cp.exp(current_logits), axis=1, keepdims=True)
            residual = Y_target - probs

            # 2. Scan for Best Math (The Technique)
            if cycle % 3 == 0 or cycle == 1:
                af_name, af_func = self.scanner.scan(X_gpu, residual)

            # 3. BRUCE LEE TRAINING (Chaos Injection)
            # We add noise to X to simulate infinite variations
            noise_level = 0.01 + (0.05 * np.exp(-cycle/20)) # Decays over time
            X_noise = X_gpu + cp.random.normal(0, noise_level, X_gpu.shape).astype(cp.float32)

            # 4. Train Memory Unit on NOISY data, fitting TRUE residual
            # This forces the unit to find the signal in the noise

            # 4. UNIT: 1536 NEURONS (T4 Sweet Spot)
            unit = MEMORY_UNIT(1536, self.input_dim, cycle, af_func)


            # We fit on NOISY data
            unit.fit_residual(X_noise, residual, alpha=50.0) # High alpha for regularization

            # 5. Verify on CLEAN data (The Test)
            correction = unit.predict_correction(X_gpu)

            # Learning Rate (Precise strikes)
            lr = 0.2 * (0.95 ** cycle)
            temp_logits = current_logits + (correction * lr)

            temp_preds = cp.asnumpy(cp.argmax(temp_logits, axis=1))
            temp_acc = accuracy_score(y, temp_preds)

            # 6. Decision
            if temp_acc > best_acc:
                print(f"   [EVOLVE] Cycle {cycle}: {af_name.upper()} | Noise: {noise_level:.3f} | Acc: {temp_acc:.4%}")
                best_acc = temp_acc
                current_logits = temp_logits
                self.memory_vault.append(unit)
                patience = 0 # Reset patience
            else:
                patience += 1
                if patience % 5 == 0:
                    print(f"   [FIGHTING] Cycle {cycle}: Stagnant. (Patience {patience}/{MAX_PATIENCE})")

            if best_acc > 0.999:
                print("   [GOD MODE] 99.9% Accuracy Reached.")
                break

        print(f" > [TITAN] Training Complete. Units: {len(self.memory_vault)}. Final Training Acc: {best_acc:.4%}")

    def predict(self, X):
        # 1. Base
        base_probs = self.base_model.predict_proba(X)
        current_logits = cp.asarray(np.log(base_probs + 1e-9))
        X_gpu = cp.asarray(X, dtype=cp.float32)

        # 2. Infinite Memory Recall
        # The units are robust because they were trained on noise
        cycle = 0
        for unit in self.memory_vault:
            cycle += 1
            correction = unit.predict_correction(X_gpu)
            lr = 0.2 * (0.95 ** cycle)
            current_logits += (correction * lr)

        return cp.asnumpy(cp.argmax(current_logits, axis=1))



# ==========================================
# 3. THE ARENA ENGINE
# ==========================================
# ==========================================
# 3. THE ARENA ENGINE (UPDATED: VERBOSE LOADER)
# ==========================================
# ==========================================
# 3. THE ARENA ENGINE (Standardized Protocol)
# ==========================================
def TITAN_VS_GOLIATHS(data_id, name):
    print(f"\n{'='*100}")
    print(f" >>> PARADIGM SHIFT PROTOCOL: {name.upper()} (ID: {data_id})")
    print(f"{'='*100}")

    try:
        data = fetch_openml(data_id=data_id, as_frame=True, parser='pandas')
        X_raw, y_raw = data.data, data.target
        if X_raw.isnull().values.any(): X_raw = X_raw.fillna(0)

        le = LabelEncoder()
        y = le.fit_transform(y_raw)
        X = X_raw.values.astype(np.float32) if hasattr(X_raw, 'values') else X_raw.astype(np.float32)

        scaler_qt = QuantileTransformer(output_distribution='normal', random_state=42)
        X_qt = scaler_qt.fit_transform(X)

        # Use FULL data for Cross-Validation (No lucky test splits)
        print(f" > [SYSTEM] Data Ready. Shape: {X_qt.shape}. Mode: 5-FOLD CROSS-VALIDATION")

    except Exception as e:
        print(f" [ERROR] {e}")
        return

    results = []

    # --- PHASE 1: STATIC MODELS (The Old Guard) ---
    print("\n >>> PHASE 1: STATIC MODELS (5-Fold CV)")
    models = [
        ("RandomForest", RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
        ("ExtraTrees", ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
        ("XGBoost", XGBClassifier(n_estimators=200, eval_metric='logloss', n_jobs=-1, random_state=42)),
        ("LightGBM", LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1, verbose=-1))
    ]

    best_static_acc = 0.0

    for n, m in models:
        t0 = time.time()
        # WE USE CROSS_VAL_SCORE TO GET THE TRUE "STRENGTH"
        scores = cross_val_score(m, X_qt, y, cv=5, n_jobs=-1)
        acc = np.mean(scores)

        if acc > best_static_acc: best_static_acc = acc

        print(f"  > {n:<15} | CV Acc: {acc:.4%} | Time: {time.time()-t0:.2f}s")
        results.append({"Model": n, "Accuracy": acc})

    # --- PHASE 2: TITAN CHIMERA (The Awakening) ---
    print("\n >>> PHASE 2: TITAN CHIMERA")
    t0 = time.time()

    # We create a wrapper to fit the API, but we manually run the protocol to match Phase 1 logic
    titan = TITAN_INF()

    # 1. Run the Tournament (same as Phase 1 basically, but selecting the host)
    base_model = titan._tournament_selection(X_qt, y)

    # 2. Train and Evolve on Folds to get a comparable CV score
    print(f" > [TITAN] Calculating Infinite Memory CV Score...")

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for train_idx, val_idx in skf.split(X_qt, y):
        X_t, X_v = X_qt[train_idx], X_qt[val_idx]
        y_t, y_v = y[train_idx], y[val_idx]

        # Create a fresh Titan for this fold
        fold_titan = TITAN_INF()
        fold_titan.base_model = base_model # Force it to use the Champion
        fold_titan.base_model.fit(X_t, y_t)

        # Get Base Probs
        try: base_probs = fold_titan.base_model.predict_proba(X_t)
        except: continue

        # Run Evolution (Fast Mode for CV)
        fold_titan._fit_evolution(X_t, y_t, base_probs)

        # Predict on Validation
        final_pred = fold_titan.predict(X_v)
        fold_scores.append(accuracy_score(y_v, final_pred))

    titan_acc = np.mean(fold_scores)
    print(f"  > TITAN INF       | CV Acc: {titan_acc:.4%} | Time: {time.time()-t0:.2f}s")
    results.append({"Model": "** TITAN INF **", "Accuracy": titan_acc})

    # --- FINAL INTELLIGENCE REPORT ---
    df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False)
    print(f"\n{'='*100}")
    print(f" {'RANK':<5} | {'MODEL':<20} | {'ACCURACY':<10} | {'ERROR':<10}")
    print(f"{'-'*100}")
    for i, r in enumerate(df.itertuples(), 1):
        print(f" #{i:<5} | {r.Model:<20} | {r.Accuracy:.4%} | {1-r.Accuracy:.4%}")
    print(f"{'='*100}\n")

# --- EXECUTE PROTOCOL ---
if __name__ == "__main__":
    print('Active')

>>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.
Active


In [ ]:
import numpy as np
import sys
import warnings
import time
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.datasets import fetch_openml

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# --- GPU ACTIVATION ---
try:
    import cupy as cp
    print(">>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.")
    cp.random.seed(42)
except ImportError:
    print(">>> [FATAL] GPU REQUIRED FOR TITAN.")
    sys.exit()

warnings.filterwarnings("ignore")

# ==========================================
# 1. THE MEMORY UNIT (The Synapse)
# ==========================================
# ==========================================
# 1. THE MEMORY UNIT (UPDATED: GOD MODE)
# ==========================================
class MEMORY_UNIT:
    def __init__(self, n_neurons, input_dim, seed, activation_func):
        cp.random.seed(seed)
        self.n_neurons = n_neurons
        self.activation = activation_func

        # RAGE INITIALIZATION: Wider variance for aggressive capture
        self.W_in = cp.random.uniform(-1.5, 1.5, (n_neurons, input_dim)).astype(cp.float32)

        # Chaos Cortex: Sparse but powerful connections
        self.W_cortex = cp.random.normal(0, 1.2 / np.sqrt(n_neurons), (n_neurons, n_neurons)).astype(cp.float32)

        # Spectral Stabilization (Critical for Infinite Recursion)
        try:
            u, s, v = cp.linalg.svd(self.W_cortex)
            self.W_cortex = self.W_cortex / (s[0] + 1e-6) * 0.95 # Keep it just below chaos
        except: pass
        self.readout = None

    def process(self, X_gpu):
        state = cp.zeros((X_gpu.shape[0], self.n_neurons), dtype=cp.float32)
        # INFINITE RECURSION: Increased to 10 for deeper reasoning
        for _ in range(2):
            pre = (X_gpu @ self.W_in.T) + (state @ self.W_cortex.T)
            state = self.activation(pre) + 0.1 * state # Residual connection
        return state

    def fit_residual(self, X_gpu, Residual_gpu, alpha=0.5):
        # Sniper Alpha: 0.5 allows extreme overfitting to the residual (The Fix)
        H = self.process(X_gpu)
        I = cp.eye(self.n_neurons, dtype=cp.float32)
        self.readout = cp.linalg.solve(H.T @ H + alpha*I, H.T @ Residual_gpu)

    def predict_correction(self, X_gpu):
        H = self.process(X_gpu)
        return H @ self.readout

    def predict(self, X):
        base_probs = self.base_model.predict_proba(X)
        current_logits = cp.asarray(np.log(base_probs + 1e-9))
        X_gpu = cp.asarray(X, dtype=cp.float32)

        cycle = 0
        for unit in self.memory_vault:
            cycle += 1
            correction = unit.predict_correction(X_gpu)
            lr = 0.15 / (1 + (cycle * 0.02))
            current_logits += (correction * lr)

        return cp.asnumpy(cp.argmax(current_logits, axis=1))

# ==========================================
# 3. THE ARENA ENGINE
# ==========================================
# ==========================================
# 3. THE ARENA ENGINE (Standardized Protocol - BRUCE LEE MODE)
# ==========================================
from sklearn.base import clone

def TITAN_VS_GOLIATHS(data_id, name):
    print(f"\n{'='*100}")
    print(f" >>> PARADIGM SHIFT PROTOCOL: {name.upper()} (ID: {data_id})")
    print(f"{'='*100}")

    try:
        data = fetch_openml(data_id=data_id, as_frame=True, parser='pandas')
        X_raw, y_raw = data.data, data.target
        if X_raw.isnull().values.any(): X_raw = X_raw.fillna(0)

        le = LabelEncoder()
        y = le.fit_transform(y_raw)
        X = X_raw.values.astype(np.float32) if hasattr(X_raw, 'values') else X_raw.astype(np.float32)

        scaler_qt = QuantileTransformer(output_distribution='normal', random_state=42)
        X_qt = scaler_qt.fit_transform(X)

        print(f" > [SYSTEM] Data Ready. Shape: {X_qt.shape}. Mode: 5-FOLD CROSS-VALIDATION")

    except Exception as e:
        print(f" [ERROR] {e}")
        return

    results = []

    # --- PHASE 1: STATIC MODELS (The Old Guard) ---
    print("\n >>> PHASE 1: STATIC MODELS (5-Fold CV)")
    models = [
        ("RandomForest", RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
        ("ExtraTrees", ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
        ("XGBoost", XGBClassifier(n_estimators=200, eval_metric='logloss', n_jobs=-1, random_state=42)),
        ("LightGBM", LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1, verbose=-1))
    ]

    best_static_acc = 0.0

    for n, m in models:
        t0 = time.time()
        scores = cross_val_score(m, X_qt, y, cv=2, n_jobs=-1)
        acc = np.mean(scores)
        if acc > best_static_acc: best_static_acc = acc
        print(f"  > {n:<15} | CV Acc: {acc:.4%} | Time: {time.time()-t0:.2f}s")
        results.append({"Model": n, "Accuracy": acc})

    # --- PHASE 2: TITAN CHIMERA (The Awakening) ---
    print("\n >>> PHASE 2: TITAN CHIMERA")
    t0 = time.time()

    titan = TITAN_INF()
    # 1. Select the Best Physical Form (e.g., ExtraTrees)
    champion_template = titan._tournament_selection(X_qt, y)

    print(f" > [TITAN] Calculating Infinite Memory CV Score...")

    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    fold_scores = []

    for train_idx, val_idx in skf.split(X_qt, y):
        X_t, X_v = X_qt[train_idx], X_qt[val_idx]
        y_t, y_v = y[train_idx], y[val_idx]

        # Create a fresh Titan for this fold
        fold_titan = TITAN_INF()
        fold_titan.n_classes = len(np.unique(y))
        fold_titan.input_dim = X_t.shape[1]

        # Clone the champion architecture
        fold_titan.base_model = clone(champion_template)

        # --- THE BRUCE LEE FIX ---
        # Instead of fitting and predicting (which gives 100% fake accuracy),
        # we generate "Honest" residuals using Cross-Validation inside the training set.
        # This reveals the REAL errors (e.g., 86% acc) so Titan knows what to fight.
        try:
            honest_probs = cross_val_predict(
                fold_titan.base_model,
                X_t, y_t,
                cv=2,
                method='predict_proba',
                n_jobs=-1
            )
        except:
            # Fallback for small datasets
            fold_titan.base_model.fit(X_t, y_t)
            honest_probs = fold_titan.base_model.predict_proba(X_t)

        # NOW we fit the model for real so it can predict later
        fold_titan.base_model.fit(X_t, y_t)

        # Run Evolution using the HONEST residuals (The "Pain")
        fold_titan._fit_evolution(X_t, y_t, honest_probs)

        # Predict on Validation
        final_pred = fold_titan.predict(X_v)
        score = accuracy_score(y_v, final_pred)
        fold_scores.append(score)

    titan_acc = np.mean(fold_scores)
    print(f"  > TITAN INF       | CV Acc: {titan_acc:.4%} | Time: {time.time()-t0:.2f}s")
    results.append({"Model": "** TITAN INF **", "Accuracy": titan_acc})

    # --- FINAL INTELLIGENCE REPORT ---
    df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False)
    print(f"\n{'='*100}")
    print(f" {'RANK':<5} | {'MODEL':<20} | {'ACCURACY':<10} | {'ERROR':<10}")
    print(f"{'-'*100}")
    for i, r in enumerate(df.itertuples(), 1):
        print(f" #{i:<5} | {r.Model:<20} | {r.Accuracy:.4%} | {1-r.Accuracy:.4%}")
    print(f"{'='*100}\n")

if __name__ == "__main__":
    print("Active")

>>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.
Active


In [ ]:
TITAN_VS_GOLIATHS(1494, "QSAR Biodegradation")


 >>> PARADIGM SHIFT PROTOCOL: QSAR BIODEGRADATION (ID: 1494)
 > [SYSTEM] Data Ready. Shape: (1055, 41). Mode: 5-FOLD CROSS-VALIDATION

 >>> PHASE 1: STATIC MODELS (5-Fold CV)
  > RandomForest    | CV Acc: 83.9793% | Time: 3.14s
  > ExtraTrees      | CV Acc: 85.1159% | Time: 0.62s
  > XGBoost         | CV Acc: 84.3585% | Time: 0.39s
  > LightGBM        | CV Acc: 83.9797% | Time: 5.37s

 >>> PHASE 2: TITAN CHIMERA
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
 > [VICTORY] Champion Selected: EXTRATREES (Base Acc: 85.9687%)
 > [TITAN] Calculating Infinite Memory CV Score...
 > [TITAN] Dojo Opened. Base Accuracy: 86.1480%
 > [STRATEGY] Bruce Lee Mode: Infinite Variations (Noise Injection)
   [EVOLVE] Cycle 1: COS_PHI_1.0 | Noise: 0.058 | Acc: 87.4763%
   [EVOLVE] Cycle 2: COS_PHI_1.0 | Noise: 0.055 | Acc: 88.8046%
   [EVOLVE] Cycle 3: SIN_PHI_2.0 | Noise: 0.053 | Acc: 89.5636%
   [EVOLVE] Cycle 4: SIN_PHI_2.0 | Noise: 0.051 | Acc: 91.2713%
   [EVOLVE] Cycle 5: SIN_PHI_2.0 | 

In [ ]:
TITAN_VS_GOLIATHS(1479, "Hill Valley")


 >>> PARADIGM SHIFT PROTOCOL: HILL VALLEY (ID: 1479)
 > [SYSTEM] Data Ready. Shape: (1212, 100). Mode: 5-FOLD CROSS-VALIDATION

 >>> PHASE 1: STATIC MODELS (5-Fold CV)
  > RandomForest    | CV Acc: 55.1980% | Time: 4.66s
  > ExtraTrees      | CV Acc: 55.6106% | Time: 1.48s
  > XGBoost         | CV Acc: 55.3630% | Time: 3.36s
  > LightGBM        | CV Acc: 55.6106% | Time: 10.81s

 >>> PHASE 2: TITAN CHIMERA
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
 > [VICTORY] Champion Selected: EXTRATREES (Base Acc: 56.1056%)
 > [TITAN] Calculating Infinite Memory CV Score...
 > [TITAN] Dojo Opened. Base Accuracy: 54.2904%
 > [STRATEGY] Bruce Lee Mode: Infinite Variations (Noise Injection)
   [EVOLVE] Cycle 1: MISH | Noise: 0.058 | Acc: 56.1056%
   [EVOLVE] Cycle 2: MISH | Noise: 0.055 | Acc: 56.9307%
   [EVOLVE] Cycle 3: SIN_PHI_1.0 | Noise: 0.053 | Acc: 59.7360%
   [EVOLVE] Cycle 4: SIN_PHI_1.0 | Noise: 0.051 | Acc: 60.5611%
   [EVOLVE] Cycle 5: SIN_PHI_1.0 | Noise: 0.049 | Acc: 

In [ ]:
TITAN_VS_GOLIATHS(40499, "Texture Analysis")

In [ ]:
TITAN_VS_GOLIATHS(1471, "EEG Eye State")

In [ ]:
TITAN_VS_GOLIATHS(1489, "Phoneme Acoustics")


 >>> PARADIGM SHIFT PROTOCOL: PHONEME ACOUSTICS (ID: 1489)
 > [SYSTEM] Data Ready. Shape: (5404, 5). Mode: 5-FOLD CROSS-VALIDATION

 >>> PHASE 1: STATIC MODELS (5-Fold CV)
  > RandomForest    | CV Acc: 88.9896% | Time: 2.04s
  > ExtraTrees      | CV Acc: 89.8594% | Time: 1.03s
  > XGBoost         | CV Acc: 88.4530% | Time: 0.30s
  > LightGBM        | CV Acc: 88.3420% | Time: 3.04s

 >>> PHASE 2: TITAN CHIMERA
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
 > [VICTORY] Champion Selected: EXTRATREES (Base Acc: 90.6366%)
 > [TITAN] Calculating Infinite Memory CV Score...
 > [TITAN] Dojo Opened. Base Accuracy: 88.0459%
 > [STRATEGY] Bruce Lee Mode: Infinite Variations (Noise Injection)
   [EVOLVE] Cycle 1: SIN_PHI_1.618 | Noise: 0.058 | Acc: 89.0822%
   [EVOLVE] Cycle 2: SIN_PHI_1.618 | Noise: 0.055 | Acc: 90.1184%
   [EVOLVE] Cycle 3: SIN_PHI_8.0 | Noise: 0.053 | Acc: 90.3035%
   [EVOLVE] Cycle 4: SIN_PHI_8.0 | Noise: 0.051 | Acc: 90.4515%
   [EVOLVE] Cycle 5: SIN_PHI_8.0 |

>>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.

 >>> PARADIGM SHIFT PROTOCOL: HILL VALLEY (ID: 1479)
 > [SYSTEM] Loading OpenML ID: 1479...
 >>> PHASE 1: STATIC MODEL EVALUATION
  > ExtraTrees Acc: 56.7901% (1.06s)

 >>> PHASE 2: TITAN INF (ETERNAL DOJO)
 > [TITAN] Initializing Base Architecture...
 > [TITAN] Generating Truth Gap (Residuals)...
 > [TITAN] Dojo Opened. Starting Acc: 100.0000%
 > [STRATEGY] Protocol: 'Torture Training'. No Stagnation Allowed.
 > [TITAN] Evolution Complete. Final Acc: 100.0000%

 >>> FINAL REPORT: Hill Valley
 TITAN INF  : 57.6132%
 ExtraTrees : 56.7901%
 >>> VICTORY MARGIN: +0.8230%
